In [1]:
%cd hw4/hw4-code/part-2-code

/teamspace/studios/this_studio/hw4/hw4-code/part-2-code


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from transformers import T5TokenizerFast
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
import numpy as np

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
train_nl = [l.strip() for l in open("data/train.nl") if l.strip()]
train_sql = [l.strip() for l in open("data/train.sql") if l.strip()]

print("Train NL examples:", len(train_nl))
print("Train SQL examples:", len(train_sql))


Train NL examples: 4225
Train SQL examples: 4225


In [4]:
dev_nl = [l.strip() for l in open("data/dev.nl") if l.strip()]
dev_sql = [l.strip() for l in open("data/dev.sql") if l.strip()]

print("Dev NL examples:", len(dev_nl))
print("Dev SQL examples:", len(dev_sql))


Dev NL examples: 466
Dev SQL examples: 466


In [5]:
train_nl_lens = [len(tokenizer.encode(x)) for x in train_nl]
print("Mean NL token length (TRAIN):", np.mean(train_nl_lens))

Mean NL token length (TRAIN): 18.097278106508877


In [6]:
dev_nl_lens = [len(tokenizer.encode(x)) for x in dev_nl]
print("Mean NL token length (DEV):", np.mean(dev_nl_lens))


Mean NL token length (DEV): 18.070815450643778


In [7]:
train_sql_lens = [len(tokenizer.encode(x)) for x in train_sql]
print("Mean SQL token length (TRAIN):", np.mean(train_sql_lens))


Mean SQL token length (TRAIN): 217.37254437869822


In [8]:
dev_sql_lens = [len(tokenizer.encode(x)) for x in dev_sql]
print("Mean SQL token length (DEV):", np.mean(dev_sql_lens))


Mean SQL token length (DEV): 211.05364806866953


In [9]:
nl_vocab_train = set()
for x in train_nl:
    nl_vocab_train.update(tokenizer.encode(x))

print("Natural language vocab size (TRAIN):", len(nl_vocab_train))


Natural language vocab size (TRAIN): 792


In [10]:
nl_vocab_dev = set()
for x in dev_nl:
    nl_vocab_dev.update(tokenizer.encode(x))

print("Natural language vocab size (DEV):", len(nl_vocab_dev))


Natural language vocab size (DEV): 466


In [11]:
sql_vocab_train = set()
for x in train_sql:
    sql_vocab_train.update(tokenizer.encode(x))

print("SQL vocab size (TRAIN):", len(sql_vocab_train))


SQL vocab size (TRAIN): 556


In [12]:
sql_vocab_dev = set()
for x in dev_sql:
    sql_vocab_dev.update(tokenizer.encode(x))

print("SQL vocab size (DEV):", len(sql_vocab_dev))


SQL vocab size (DEV): 396


In [13]:
from load_data import T5Dataset

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
dset_train = T5Dataset("data", "train")
dset_dev   = T5Dataset("data", "dev")

In [15]:
def mean_length(input_ids_list):
    return np.mean([len(x) for x in input_ids_list])

print("AFTER PREPROCESSING")
print("Train mean NL length:", mean_length(dset_train.encoder_ids))
print("Dev mean NL length:", mean_length(dset_dev.encoder_ids))

print("Train mean SQL length:", mean_length(dset_train.decoder_ids))
print("Dev mean SQL length:", mean_length(dset_dev.decoder_ids))



AFTER PREPROCESSING
Train mean NL length: 18.097278106508877
Dev mean NL length: 18.070815450643778
Train mean SQL length: 217.37254437869822
Dev mean SQL length: 211.05364806866953


In [16]:
def vocab_size_from_ids(list_of_id_lists):
    vocab = set()
    for ids in list_of_id_lists:
        vocab.update(ids)
    return len(vocab)

print("Train NL vocab size:", vocab_size_from_ids(dset_train.encoder_ids))
print("Dev   NL vocab size:", vocab_size_from_ids(dset_dev.encoder_ids))

print("Train SQL vocab size:", vocab_size_from_ids(dset_train.decoder_ids))
print("Dev   SQL vocab size:", vocab_size_from_ids(dset_dev.decoder_ids))

Train NL vocab size: 792
Dev   NL vocab size: 466
Train SQL vocab size: 556
Dev   SQL vocab size: 396


In [17]:
!python evaluate.py --predicted_sql results/t5_ft_dev.sql --predicted_records records/t5_ft_dev.pkl --development_sql data/dev.sql --development_records records/ground_truth_dev.pkl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Traceback (most recent call last):
  File "/teamspace/studios/this_studio/hw4/hw4-code/part-2-code/evaluate.py", line 15, in <module>
    _, _, record_f1, _ = compute_metrics(args.dev_sql, args.pred_sql, args.dev_records, args.pred_records)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/teamspace/studios/this_studio/hw4/hw4-code/part-2-code/utils.py", line 32, in compute_metrics
    model_qs, model_records, model_error_msgs = load_queries_and_records(model_path, model_query_records)
                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/teamspace/studios/this_studio/hw4/hw4-code/part-2-code/utils.py", line 52, in load_queries_and_records
    read_qs = read_queries(sql_path)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/teamspace/studios/this_studio/hw4/hw4-code/part-2-code/utils.py", line 83, in read_queries
    with open(sql_path, 'r') as f:
         ^^^^^^^

In [18]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install numpy==2.0.0
!pip install transformers==4.40.0
!pip install tokenizers==0.19.1
!pip install accelerate==0.29.3
!pip install nltk==3.8.1
!pip install tqdm==4.66.1
!pip install matplotlib==3.8.0
!pip install plotly==5.18.0
!pip install seaborn==0.13.2
!pip install sentencepiece==0.2.0
!pip install wandb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu121


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-2.0.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.0.0 which is incompatible.
scikit-learn 1.3.2 requires numpy<2.0,>=1.17.3, but you have numpy 2.0.0 which is incompatible.
pandas 2.1.4 requires numpy<2,>=1.26.0; python_version >= "3.12", but you have numpy 2.0.0 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
pip install torch==2.5.1 torchvision --index-url https://download.pytorch.org/whl/cu121


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python -u train_t5.py \
  --finetune \
  --learning_rate 1e-3 \
  --max_n_epochs 5 \
  --patience_epochs 2 \
  --batch_size 8 \
  --test_batch_size 8



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|█████████████████████████████████████████| 529/529 [00:42<00:00, 12.34it/s]
Epoch 0: Average train loss was 0.3104625663684602
100%|███████████████████████████████████████████| 59/59 [00:18<00:00,  3.25it/s]
466it [00:00, 11716.85it/s]
Epoch 0: Dev loss: 0.06809785079107762, Record F1: 0.11802575048283262, Record EM: 0.11802575107296137, SQL EM: 0.0
Epoch 0: SQL error rate = 100.00%
100%|█████████████████████████████████████████| 529/529 [00:42<00:00, 12.39it/s]
Epoch 1: Average train loss was 0.06917746367596801
100%|█████████

In [2]:
!ls results
!ls records


llm_test.sql		     t5_ft_ft_experiment_test.sql  t5_scr_test.sql
t5_ft_ft_experiment_dev.sql  t5_ft_test.sql
ground_truth_dev.pkl  t5_ft_ft_experiment_dev.pkl   t5_ft_test.pkl
llm_test.pkl	      t5_ft_ft_experiment_test.pkl  t5_scr_test.pkl


In [3]:
ls -R


.:
README.md*    evaluate.py*         records/           train_t5.py*
__pycache__/  load_data.py*        requirements.txt*  utils.py*
checkpoints/  prompting.py*        results/
data/         prompting_utils.py*  t5_utils.py*

./__pycache__:
load_data.cpython-312.pyc  t5_utils.cpython-312.pyc  utils.cpython-312.pyc

./checkpoints:
ft_experiments/

./checkpoints/ft_experiments:
experiment/

./checkpoints/ft_experiments/experiment:
best.pt  last.pt

./data:
alignment.txt*  dev.sql*             flight_database.schema*  train.nl*
dev.nl*         flight_database.db*  test.nl*                 train.sql*

./records:
ground_truth_dev.pkl*  t5_ft_ft_experiment_dev.pkl   t5_ft_test.pkl*
llm_test.pkl*          t5_ft_ft_experiment_test.pkl  t5_scr_test.pkl*

./results:
llm_test.sql*                t5_ft_ft_experiment_test.sql  t5_scr_test.sql*
t5_ft_ft_experiment_dev.sql  t5_ft_test.sql*
